In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os
import time

from t2_functions import partition_ids

In [2]:
import sys

path = os.getcwd()
print(path)
os.chdir('..')
print(os.getcwd())
dataset_t1 = torch.load("task_1_modelstealing/data/ModelStealingPub.pt")

sys.path.append(os.path.join(os.getcwd(), "task_2_sybilattack/"))

from endpoints.requests import sybil, sybil_reset


/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack
/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam


In [3]:
os.chdir(path)
print(os.getcwd())
dataset = torch.load("data/SybilAttack.pt")

/home/janek/Documents/Hackaton/ensembleAI-ScoutTeam/task_2_sybilattack


In [4]:
ids = np.array(dataset.ids)
print(len(ids))
print(ids[:10])
binned_ids = partition_ids(ids, main_bin_num=10, train=0.1, test=0.9)

20000
[101031   8526  43127 191394 298792 121086 149475 102605 163605 101855]
bin_size: 2000
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800
train: 200, test: 1800


In [36]:
task = 'affine'

sybil_reset(home_or_defense='home', binary_or_affine=task)
sybil_reset(home_or_defense='defense', binary_or_affine=task)


Request ok
{'msg': 'Successful sybil affine reset home'}
Request ok
{'msg': 'Successful sybil affine reset defense'}


In [37]:
ids_train, ids_test = binned_ids[0]
print(len(ids_train))
print(len(ids_test))
A_train_reps = sybil(ids=ids_train,
                 home_or_defense='home',
                 binary_or_affine=task)

B_train_reps = sybil(ids=ids_train,
                 home_or_defense='defense',
                 binary_or_affine=task)
print(f"A train reps: {len(A_train_reps)}")

A_test_reps = sybil(ids=ids_test,
                 home_or_defense='home',
                 binary_or_affine=task)

B_test_reps = sybil(ids=ids_test,
                 home_or_defense='defense',
                 binary_or_affine=task)
print(f"A test reps: {len(A_test_reps)}")


200
1800
A train reps: 200
A test reps: 1800


In [7]:
class RepresentationsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]
        return torch.tensor(x), torch.tensor(y)


In [8]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super(Linear, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [10]:
def validate(criterion, loader, net):
    net.eval()

    with torch.no_grad():
        true = []
        pred = []
        for x, y in loader:
            y_hat = net(x)
            true.append(y.numpy())
            pred.append(y_hat.numpy())

        true = torch.tensor(np.concatenate(true, axis=0))
        pred = torch.tensor(np.concatenate(pred, axis=0))

        loss = criterion(pred, true)

    return loss


In [11]:
def l1_reg(model, reg_lambda):
    l1_regularization = torch.tensor(0., device=model.parameters().__next__().device)
    for param in model.parameters():
        l1_regularization += torch.norm(param, p=1)
    return reg_lambda * l1_regularization


In [75]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train(epochs, optim, criterion, regularise, trainloader, valloader, net, empty_net, reg_lambda=None):
    best_val_loss = np.inf
    best_net = empty_net

    for epoch in range(epochs):  # loop over the dataset multiple times
        train_loss = 0

        for iter, (x, y) in enumerate(tqdm(trainloader)):
            net.train()
            optim.zero_grad()

            y_hat = net(x)
            loss = criterion(y_hat, y)
            if reg_lambda is not None:
                loss += regularise(net, reg_lambda)
            loss.backward()
            optim.step()

            batch_loss = loss.item()
            train_loss += batch_loss

        train_loss /= len(trainloader)
        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}")

        net.eval()

        val_loss = validate(criterion, valloader, net)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_net.load_state_dict(net.state_dict())

        print(f"Epoch [{epoch + 1}/{epochs}], Val Loss: {val_loss:.4f}")

    return net, best_net



In [83]:
criterion = nn.MSELoss()
batch_size = 32
epochs = 100
lr = 0.001


In [84]:
def lin_augment_affine(xs, ys, new_for_each_pair=3):
    new_x = []
    new_y = []
    for i, (x1, y1) in tqdm(enumerate(zip(xs, ys))):
        for j, (x2, y2) in enumerate(zip(xs, ys)):
            if i != j:
                weights = np.random.uniform(-1, 2, size=new_for_each_pair)
                for w in weights:
                    x_new = w*x1 + (1-w)*x2
                    y_new = w*y1 + (1-w)*y2
                    new_x.append(x_new)
                    new_y.append(y_new)
    new_x = np.array(new_x)
    new_y = np.array(new_y)
    return np.concatenate((xs, new_x), axis=0), np.concatenate((ys, new_y), axis=0)


In [85]:
A_train_reps_tensor = torch.tensor(A_train_reps, dtype=torch.float32)
B_train_reps_tensor = torch.tensor(B_train_reps, dtype=torch.float32)
A_train_aug, B_train_aug = lin_augment_affine(A_train_reps_tensor, B_train_reps_tensor)

0it [00:00, ?it/s]

200it [00:09, 20.26it/s]


In [86]:
train_dataset = RepresentationsDataset(x=A_train_reps, y=B_train_reps)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = RepresentationsDataset(x=A_test_reps, y=B_test_reps)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [87]:
lin_net = Linear(384, 384)
lin_empty_net = Linear(384, 384)

optim = torch.optim.Adam(lr=lr, params=lin_net.parameters())

lin_last_net, lin_best_net = train(epochs, optim, criterion, l1_reg, train_loader, test_loader, lin_net, lin_empty_net, reg_lambda=0.0001)

100%|██████████| 7/7 [00:00<00:00, 67.91it/s]


Epoch [1/100], Train Loss: 2.4230
Epoch [1/100], Val Loss: 1.1006


100%|██████████| 7/7 [00:00<00:00, 61.56it/s]

Epoch [2/100], Train Loss: 1.2805


Epoch [2/100], Val Loss: 0.8469


...

Epoch [99/100], Train Loss: 0.1194


Epoch [99/100], Val Loss: 0.1178


100%|██████████| 7/7 [00:00<00:00, 38.74it/s]

Epoch [100/100], Train Loss: 0.1199


Epoch [100/100], Val Loss: 0.1177


In [65]:
mlp = MLP(384, 32, 384)
empty_mpl = MLP(384, 32, 384)

optim = torch.optim.Adam(lr=lr, params=mlp.parameters())

mlp_last_net, mlp_best_net = train(epochs, optim, criterion, l1_reg, train_loader, test_loader, mlp, empty_mpl, reg_lambda=0.0)

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
test_loss_lin = validate(criterion, test_loader, lin_last_net)
print(f"lin test: {test_loss_lin}")

test_loss_mlp = validate(criterion, test_loader, mlp_last_net)
print(f"lin test: {test_loss_mlp}")
